# Combined Electricity & Weather EDA

This notebook consolidates the global exploratory analysis from `data_loader.py` with the per-meter tooling from `per_meter_eda.py`, and extends it with intra-meter weekly diagnostics.


## Notebook Guide

- Configure `DATA_PATH` below to point at the merged electricity/weather dataset.
- Helper utilities keep track of column naming differences (upper vs lower case).
- Run the notebook sequentially so that downstream cells reuse shared tables.


In [16]:
import numpy as np
import pandas as pd
import re
from pandas.api.types import DatetimeTZDtype

from pathlib import Path
from typing import Dict, Iterable, List, Optional

import matplotlib.pyplot as plt

try:
    import seaborn as sns
except ImportError:
    sns = None

try:
    from IPython.display import display
except ImportError:
    display = print  # type: ignore[arg-type]

if sns is not None:
    sns.set_theme(style="whitegrid")

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12
pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 4)


In [ ]:
# Update this list if your dataset lives elsewhere.
DEFAULT_DATA_PATHS = [
    Path("/Users/jovanpopovic/Documents/Uni/Honours 2025/honours_project/datasets/original_data.csv"),
]

DATA_PATH = next((path for path in DEFAULT_DATA_PATHS if path.exists()), None)

# if DATA_PATH is None:
#     DATA_PATH = Path("/absolute/path/to/merged_electricity_weather.parquet")
#     print("Set DATA_PATH to the merged electricity/weather dataset before running the next cell.")
# else:
#     print(f"Using dataset at: {DATA_PATH.resolve()}")


Set DATA_PATH to the merged electricity/weather dataset before running the next cell.


In [15]:
DROP_CANDIDATES = [
    "ref",
    "row",
    "quarter",
    "aggregate_year",
    "aggregate_month",
    "aggregate_day",
    "aggregate_hour",
    "period_over_which_rainfall_was_measured_days",
    "days_of_accumulation_of_maximum_temperature",
    "days_of_accumulation_of_minimum_temperature",
]

COLUMN_ALIASES: Dict[str, Iterable[str]] = {
    "meter": ("meter_ui", "meter", "meter_id"),
    "nmi": ("nmi_ui", "nmi_id"),
    "delivered": ("delivered_value", "DELIVERED_VALUE"),
    "daily_energy": (
        "daily_energy_usage",
        "DAILY_ENERGY_USAGE",
        "Daily Energy Usage",
    ),
    "received": ("received_value", "RECEIVED_VALUE"),
    "power_zero": ("power_zero", "POWER_ZERO"),
    "daily_energy_zero": ("daily_energy_zero", "DAILY_ENERGY_ZERO"),
    "error_check_hour": ("error_check_hour", "Error Check Hour"),
    "error_check_day": ("error_check_day", "Error Check Day"),
    "quarter": ("quarter", "Quarter"),
    "aggregate_date": ("aggregate_date", "Aggregate Date"),
    "date": ("date", "Date"),
    "time": ("time", "Time"),
    "timestamp": ("timestamp", "Timestamp"),
}


def resolve_column(df: pd.DataFrame, *candidates: Optional[str]) -> Optional[str]:
    lookup = {col.lower(): col for col in df.columns}
    for candidate in candidates:
        if candidate is None:
            continue
        column = lookup.get(candidate.lower())
        if column is not None:
            return column
    return None





def _guess_dayfirst(series: pd.Series, sample_size: int = 20) -> bool:
    sample = series.dropna()
    if sample.empty:
        return False
    if pd.api.types.is_datetime64_any_dtype(sample):
        return False
    sample_str = sample.astype(str).str.strip()
    sample_str = sample_str[sample_str != ""].head(sample_size)
    if sample_str.empty:
        return False
    for value in sample_str:
        tokens = [tok for tok in re.split(r"[^0-9]", value) if tok]
        if len(tokens) >= 3:
            first = tokens[0]
            last = tokens[-1]
            if len(last) == 4 and len(first) <= 2:
                return True
    return False
def _optimize_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    opt = df.copy()

    obj_cols = opt.select_dtypes(include=["object", "string"]).columns
    for col in obj_cols:
        series = opt[col]
        name = col.lower()

        if "date" in name or "timestamp" in name:
            dayfirst = _guess_dayfirst(series)
            converted = pd.to_datetime(series, errors="coerce", dayfirst=dayfirst)
            if isinstance(converted.dtype, DatetimeTZDtype):
                converted = converted.dt.tz_localize(None)
            if converted.notna().mean() > 0.9:
                opt[col] = converted
                continue

        numeric = pd.to_numeric(series, errors="coerce")
        if numeric.notna().mean() > 0.9:
            if (numeric.dropna() % 1 == 0).all():
                opt[col] = pd.to_numeric(series, errors="coerce", downcast="integer")
            else:
                opt[col] = pd.to_numeric(series, errors="coerce", downcast="float")
            continue

        nunique = series.nunique(dropna=True)
        if nunique < 50 or (nunique / max(len(series), 1) < 0.5):
            opt[col] = series.astype("category")

    int_cols = opt.select_dtypes(include=["int64", "int32", "Int64", "Int32"]).columns
    if len(int_cols) > 0:
        opt[int_cols] = opt[int_cols].apply(pd.to_numeric, downcast="integer")

    float_cols = opt.select_dtypes(include=["float64", "Float64"]).columns
    if len(float_cols) > 0:
        opt[float_cols] = opt[float_cols].apply(pd.to_numeric, downcast="float")

    num_cols = opt.select_dtypes(include=[np.number]).columns
    for col in num_cols:
        series = opt[col]
        values = pd.unique(series.dropna())
        if len(values) <= 2 and set(values).issubset({0, 1, 0.0, 1.0}):
            try:
                opt[col] = series.astype("boolean") if series.isna().any() else series.astype(bool)
            except Exception:
                pass

    return opt


def load_dataset(data_path: Path, optimize: bool = True) -> pd.DataFrame:
    if not data_path.exists():
        raise FileNotFoundError(f"Dataset not found at {data_path}")

    suffix = data_path.suffix.lower()
    if suffix in {".parquet", ".pq"}:
        df = pd.read_parquet(data_path)
    elif suffix == ".csv":
        dtype_map = {
            "nmi_ui": "category",
            "meter_ui": "category",
            "date": "string",
            "time": "string",
            "error_check_day": "float32",
            "error_check_hour": "float32",
            "delivered_value": "float32",
            "daily_energy_usage": "float32",
            "received_value": "float32",
            "power_zero": "float32",
            "daily_energy_zero": "float32",
            "rainfall_amount_millimetres": "float32",
            "period_over_which_rainfall_was_measured_days": "float32",
            "maximum_temperature_degree_c": "float32",
            "days_of_accumulation_of_maximum_temperature": "float32",
            "minimum_temperature_degree_c": "float32",
            "days_of_accumulation_of_minimum_temperature": "float32",
            "daily_global_solar_exposure_mj_m_m": "float32",
        }
        try:
            df = pd.read_csv(
                data_path,
                dtype=dtype_map,
                parse_dates=["aggregate_date"],
                na_values=["", " "],
            )
        except ValueError:
            df = pd.read_csv(
                data_path,
                dtype=dtype_map,
                na_values=["", " "],
            )
    else:
        raise ValueError(f"Unsupported file extension: {suffix}")

    df = df.copy()
    df.columns = [col.strip() for col in df.columns]

    if optimize:
        df = _optimize_dtypes(df)

    return df


def ensure_timestamp_column(df: pd.DataFrame, target: str = "timestamp") -> Optional[str]:
    timestamp_col = resolve_column(df, target)
    if timestamp_col is not None:
        dayfirst_guess = _guess_dayfirst(df[timestamp_col])
        df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors="coerce", dayfirst=dayfirst_guess)
        if isinstance(df[timestamp_col].dtype, DatetimeTZDtype):
            df[timestamp_col] = df[timestamp_col].dt.tz_localize(None)
        timestamp_col = timestamp_col
    else:
        date_col = resolve_column(df, "date", "aggregate_date")
        time_col = resolve_column(df, "time")
        if date_col and time_col:
            date_str = df[date_col].astype(str).str.strip()
            time_str = df[time_col].astype(str).str.strip()
            parsed = pd.to_datetime(
                date_str + " " + time_str,
                format="%d/%m/%Y %H:%M",
                errors="coerce",
            )
            df[target] = parsed
            if isinstance(df[target].dtype, DatetimeTZDtype):
                df[target] = df[target].dt.tz_localize(None)
            timestamp_col = target
        elif date_col:
            dayfirst_guess = _guess_dayfirst(df[date_col])
            df[target] = pd.to_datetime(
                df[date_col],
                errors="coerce",
                dayfirst=dayfirst_guess,
            )
            if isinstance(df[target].dtype, DatetimeTZDtype):
                df[target] = df[target].dt.tz_localize(None)
            timestamp_col = target
        else:
            timestamp_col = None

    if timestamp_col:
        agg_col = resolve_column(df, "aggregate_date")
        if agg_col:
            dayfirst_guess = _guess_dayfirst(df[agg_col])
            agg_parsed = pd.to_datetime(
                df[agg_col],
                errors="coerce",
                dayfirst=dayfirst_guess,
            )
            if isinstance(agg_parsed.dtype, DatetimeTZDtype):
                agg_parsed = agg_parsed.dt.tz_localize(None)
            missing_mask = df[timestamp_col].isna()
            if missing_mask.any():
                df.loc[missing_mask, timestamp_col] = agg_parsed[missing_mask]

        if isinstance(df[timestamp_col].dtype, DatetimeTZDtype):
            df[timestamp_col] = df[timestamp_col].dt.tz_localize(None)

    return timestamp_col


def drop_known_columns(df: pd.DataFrame, protect: Optional[Iterable[str]] = None) -> pd.DataFrame:
    protect_set = set(filter(None, protect or []))
    to_drop: List[str] = []
    for candidate in DROP_CANDIDATES:
        column = resolve_column(df, candidate)
        if column and column not in protect_set:
            to_drop.append(column)
    if to_drop:
        df = df.drop(columns=sorted(set(to_drop)))
    return df


def identify_columns(df: pd.DataFrame) -> Dict[str, Optional[str]]:
    return {key: resolve_column(df, *aliases) for key, aliases in COLUMN_ALIASES.items()}




In [ ]:
def compute_meter_summaries(
    df: pd.DataFrame,
    meter_col: str,
    timestamp_col: Optional[str] = None,
    delivered_col: Optional[str] = None,
    daily_energy_col: Optional[str] = None,
    received_col: Optional[str] = None,
    nmi_col: Optional[str] = None,
    power_zero_col: Optional[str] = None,
    daily_energy_zero_col: Optional[str] = None,
) -> pd.DataFrame:
    grouped = df.groupby(meter_col, observed=True)

    named_aggs: Dict[str, pd.NamedAgg] = {}
    if nmi_col:
        named_aggs["nmi_count"] = pd.NamedAgg(column=nmi_col, aggfunc="nunique")
    if timestamp_col:
        named_aggs["start_time"] = pd.NamedAgg(column=timestamp_col, aggfunc="min")
        named_aggs["end_time"] = pd.NamedAgg(column=timestamp_col, aggfunc="max")
    if delivered_col:
        named_aggs["observation_count"] = pd.NamedAgg(column=delivered_col, aggfunc="count")
        named_aggs["delivered_mean"] = pd.NamedAgg(column=delivered_col, aggfunc="mean")
        named_aggs["delivered_median"] = pd.NamedAgg(column=delivered_col, aggfunc="median")
        named_aggs["delivered_std"] = pd.NamedAgg(column=delivered_col, aggfunc="std")
        named_aggs["delivered_min"] = pd.NamedAgg(column=delivered_col, aggfunc="min")
        named_aggs["delivered_p25"] = pd.NamedAgg(column=delivered_col, aggfunc=lambda x: x.quantile(0.25))
        named_aggs["delivered_p75"] = pd.NamedAgg(column=delivered_col, aggfunc=lambda x: x.quantile(0.75))
        named_aggs["delivered_max"] = pd.NamedAgg(column=delivered_col, aggfunc="max")
        named_aggs["delivered_sum"] = pd.NamedAgg(column=delivered_col, aggfunc="sum")
    if daily_energy_col:
        named_aggs["daily_energy_mean"] = pd.NamedAgg(column=daily_energy_col, aggfunc="mean")
        named_aggs["daily_energy_std"] = pd.NamedAgg(column=daily_energy_col, aggfunc="std")
        named_aggs["daily_energy_sum"] = pd.NamedAgg(column=daily_energy_col, aggfunc="sum")
    if received_col:
        named_aggs["received_mean"] = pd.NamedAgg(column=received_col, aggfunc="mean")
        named_aggs["received_sum"] = pd.NamedAgg(column=received_col, aggfunc="sum")
    if power_zero_col:
        named_aggs["power_zero_rate"] = pd.NamedAgg(column=power_zero_col, aggfunc="mean")
    if daily_energy_zero_col:
        named_aggs["daily_energy_zero_rate"] = pd.NamedAgg(column=daily_energy_zero_col, aggfunc="mean")

    if not named_aggs:
        raise ValueError("No aggregations defined. Verify that the expected columns are present.")

    summary = grouped.agg(**named_aggs)

    if "nmi_count" in summary.columns:
        summary["nmi_count"] = summary["nmi_count"].astype("int64")
    if "observation_count" in summary.columns:
        summary["observation_count"] = summary["observation_count"].astype("int64")

    if (
        timestamp_col
        and "start_time" in summary.columns
        and "end_time" in summary.columns
        and "observation_count" in summary.columns
    ):
        summary["period_hours"] = (
            summary["end_time"] - summary["start_time"]
        ).dt.total_seconds() / 3600.0 + 1
        summary["period_days"] = summary["period_hours"] / 24.0
        summary["coverage_pct"] = (
            summary["observation_count"] / summary["period_hours"].clip(lower=1.0) * 100.0
        )

    if delivered_col and "delivered_std" in summary.columns and "delivered_mean" in summary.columns:
        summary["delivered_cv"] = (
            summary["delivered_std"] / summary["delivered_mean"]
        ).replace([np.inf, -np.inf], np.nan)

    if "power_zero_rate" in summary.columns:
        summary["power_zero_pct"] = summary.pop("power_zero_rate") * 100.0
    if "daily_energy_zero_rate" in summary.columns:
        summary["daily_energy_zero_pct"] = summary.pop("daily_energy_zero_rate") * 100.0

    return summary.sort_index()


def attach_peer_comparison(
    df: pd.DataFrame,
    summary: pd.DataFrame,
    meter_col: str,
    delivered_col: Optional[str] = None,
    daily_energy_col: Optional[str] = None,
) -> pd.DataFrame:
    summary = summary.copy()

    if delivered_col and delivered_col in df.columns and "delivered_mean" in summary.columns:
        delivered_stats = df.groupby(meter_col, observed=True)[delivered_col].agg(["sum", "count"])
        delivered_totals = delivered_stats["sum"]
        delivered_counts = delivered_stats["count"]
        all_sum = delivered_totals.sum()
        all_count = delivered_counts.sum()

        with np.errstate(divide="ignore", invalid="ignore"):
            peer_mean = (all_sum - delivered_totals) / (all_count - delivered_counts)
            peer_mean = peer_mean.replace({0.0: np.nan})
            summary["delivered_vs_peers_pct"] = (
                (summary["delivered_mean"] - peer_mean)
                / peer_mean
                * 100.0
            )

        if "delivered_sum" in summary.columns:
            summary["delivered_mean_rank"] = (
                summary["delivered_mean"].rank(ascending=False, method="dense").astype(int)
            )
            summary["delivered_sum_rank"] = (
                summary["delivered_sum"].rank(ascending=False, method="dense").astype(int)
            )

    if daily_energy_col and daily_energy_col in df.columns and "daily_energy_mean" in summary.columns:
        energy_stats = df.groupby(meter_col, observed=True)[daily_energy_col].agg(["sum", "count"])
        energy_totals = energy_stats["sum"]
        energy_counts = energy_stats["count"]
        all_energy_sum = energy_totals.sum()
        all_energy_count = energy_counts.sum()

        with np.errstate(divide="ignore", invalid="ignore"):
            peer_energy_mean = (all_energy_sum - energy_totals) / (all_energy_count - energy_counts)
            peer_energy_mean = peer_energy_mean.replace({0.0: np.nan})
            summary["daily_energy_vs_peers_pct"] = (
                (summary["daily_energy_mean"] - peer_energy_mean)
                / peer_energy_mean
                * 100.0
            )

    return summary


def hourly_usage_profile(
    df: pd.DataFrame,
    meter_id: str,
    meter_col: str,
    timestamp_col: str,
    value_col: str,
) -> pd.DataFrame:
    if timestamp_col not in df.columns:
        raise ValueError("Timestamp column not present in dataframe")
    if meter_id not in set(df[meter_col].astype(str)):
        raise ValueError(f"Meter {meter_id} not present in dataset")

    df = df.copy()
    df[meter_col] = df[meter_col].astype(str)
    df[meter_col] = df[meter_col].astype("category")
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors="coerce")

    meter_mask = df[meter_col] == str(meter_id)
    hours = df[timestamp_col].dt.hour
    meter_profile = (
        df.loc[meter_mask]
        .groupby(hours[meter_mask])[value_col]
        .agg(meter_mean="mean", meter_median="median")
    )
    peers_profile = (
        df.loc[~meter_mask]
        .groupby(hours[~meter_mask])[value_col]
        .mean()
    )

    profile = (
        meter_profile.reindex(range(24))
        .join(peers_profile.rename("peer_mean"), how="left")
    )

    with np.errstate(divide="ignore", invalid="ignore"):
        profile["lift_pct"] = (
            (profile["meter_mean"] - profile["peer_mean"])
            / profile["peer_mean"]
            * 100.0
        )

    profile.index.name = "hour"
    return profile.reset_index()


def monthly_usage_profile(
    df: pd.DataFrame,
    meter_id: str,
    meter_col: str,
    timestamp_col: str,
    value_col: str,
) -> pd.DataFrame:
    df = df.copy()
    df[meter_col] = df[meter_col].astype(str)
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors="coerce")
    df["month"] = df[timestamp_col].dt.to_period("M")

    meter_mask = df[meter_col] == str(meter_id)
    if not meter_mask.any():
        raise ValueError(f"Meter {meter_id} not present in dataset")

    meter_month = df.loc[meter_mask].groupby("month")[value_col].agg(
        meter_mean="mean",
        meter_median="median",
        meter_sum="sum",
    )
    peers_month = df.loc[~meter_mask].groupby("month")[value_col].agg(
        peer_mean="mean",
        peer_median="median",
        peer_sum="sum",
    )

    profile = meter_month.join(peers_month, how="left")
    with np.errstate(divide="ignore", invalid="ignore"):
        profile["sum_contribution_pct"] = (
            profile["meter_sum"]
            / (profile["meter_sum"] + profile["peer_sum"])
            * 100.0
        )

    return profile.reset_index()


def _finalize_plot(fig: plt.Figure, output_path: Optional[Path], show: bool) -> Optional[Path]:
    if output_path is not None:
        fig.savefig(output_path, dpi=150, bbox_inches="tight")
    if show:
        plt.show()
    plt.close(fig)
    return output_path


def generate_summary_plots(
    summary: pd.DataFrame,
    output_dir: Optional[Path] = None,
    show: bool = False,
    top_n: int = 15,
) -> List[Path]:
    if summary.empty:
        return []
    if output_dir is not None:
        output_dir.mkdir(parents=True, exist_ok=True)

    top_n = max(1, min(top_n, len(summary)))
    saved: List[Path] = []

    plot_definitions = [
        (
            "delivered_mean",
            f"Top {top_n} meters by average delivered value",
            "Average delivered value (kWh)",
        ),
        (
            "delivered_sum",
            f"Top {top_n} meters by total delivered value",
            "Total delivered value (kWh)",
        ),
        (
            "power_zero_pct",
            "Distribution of zero-power readings across meters",
            "Power zero percentage",
        ),
        (
            "coverage_pct",
            "Coverage percentage per meter",
            "Coverage percentage",
        ),
    ]

    for metric, title, xlabel in plot_definitions:
        if metric not in summary.columns:
            continue
        if sns is None and metric not in {"delivered_mean", "delivered_sum"}:
            continue

        if metric in {"delivered_mean", "delivered_sum"}:
            ordered = (
                summary.dropna(subset=[metric])
                .sort_values(metric, ascending=False)
                .head(top_n)
            )
            if ordered.empty:
                continue

            fig, ax = plt.subplots(figsize=(10, 6))
            meters = ordered.index.astype(str)
            values = ordered[metric]
            color = None
            if sns is not None:
                color = sns.color_palette("viridis", len(meters))
            ax.barh(meters, values, color=color)
            ax.set_title(title)
            ax.set_xlabel(xlabel)
            ax.set_ylabel("Meter")
            ax.invert_yaxis()
            for idx, value in enumerate(values):
                ax.text(value, idx, f" {value:,.2f}", va="center", ha="left", fontsize=8)
            fig.tight_layout()
            output_path = output_dir / f"{metric}_top_{top_n}.png" if output_dir else None
            saved_path = _finalize_plot(fig, output_path, show)
            if saved_path is not None:
                saved.append(saved_path)
        else:
            fig, ax = plt.subplots(figsize=(8, 5))
            data = summary[metric].dropna()
            if data.empty:
                plt.close(fig)
                continue
            if sns is not None:
                sns.histplot(data, bins=20, kde=True, ax=ax)
            else:
                ax.hist(data, bins=20, alpha=0.75, color="tab:blue", edgecolor="black")
            ax.set_title(title)
            ax.set_xlabel(xlabel)
            ax.set_ylabel("Count")
            fig.tight_layout()
            output_path = output_dir / f"{metric}_distribution.png" if output_dir else None
            saved_path = _finalize_plot(fig, output_path, show)
            if saved_path is not None:
                saved.append(saved_path)

    return saved


## 1. Load and Prepare Data


In [ ]:
df = load_dataset(DATA_PATH)
column_map = identify_columns(df)
timestamp_col = ensure_timestamp_column(df)

protect_columns = [
    column_map.get("meter"),
    column_map.get("nmi"),
    timestamp_col,
    column_map.get("aggregate_date"),
    column_map.get("date"),
    column_map.get("time"),
]
df = drop_known_columns(df, protect=protect_columns)

print(f"Rows: {len(df):,} | Columns: {df.shape[1]}")
display(df.head())


In [ ]:
numeric_columns = df.select_dtypes(include=[np.number, "Float32", "Float64", "Int32", "Int64", "boolean"]).columns.tolist()
categorical_columns = df.select_dtypes(include=["category", "object", "string"]).columns.tolist()

print(f"Numeric columns: {len(numeric_columns)}")
print(f"Categorical columns: {len(categorical_columns)}")

column_map["delivered"] = column_map.get("delivered") or resolve_column(df, "delivered_value")
column_map["daily_energy"] = column_map.get("daily_energy") or resolve_column(df, "daily_energy_usage")
column_map["received"] = column_map.get("received") or resolve_column(df, "received_value")

value_columns = [
    column_map.get("delivered"),
    column_map.get("daily_energy"),
    column_map.get("received"),
]
value_columns = [col for col in value_columns if col in df.columns]

print("Target candidates:", value_columns)


## 2. Global EDA (Former `data_loader.py` helpers)


In [ ]:
print("Top of data")
display(df.head())

print("
Data types")
df.info()

print("
Summary statistics (numeric)")
display(df.describe().T)


In [ ]:
missing_summary = (
    df.isna().sum()
    .to_frame(name="missing_count")
    .assign(missing_pct=lambda x: x["missing_count"] / len(df) * 100)
    .sort_values("missing_pct", ascending=False)
)
print("Missing values (sorted)")
display(missing_summary.head(20))

duplicate_count = df.duplicated().sum()
print(f"Duplicate rows: {duplicate_count}")


In [ ]:
unique_counts = df.nunique(dropna=True)
print("Unique values per column (top 20)")
display(unique_counts.sort_values(ascending=False).head(20))

error_hour_col = column_map.get("error_check_hour")
if error_hour_col and error_hour_col in df.columns:
    print(f"
Unique values in '{error_hour_col}'")
    display(df[error_hour_col].value_counts(dropna=False))
else:
    print("
No error check hour column found.")

error_day_col = column_map.get("error_check_day")
if error_day_col and error_day_col in df.columns:
    print(f"
Unique values in '{error_day_col}'")
    display(df[error_day_col].value_counts(dropna=False))


In [ ]:
received_col = column_map.get("received")
if received_col and received_col in df.columns:
    print(f"Analyzing {received_col}")
    display(df[received_col].describe())
    missing_count = df[received_col].isna().sum()
    print(f"Missing values: {missing_count} ({missing_count / len(df) * 100:.2f}%)")
    display(df[received_col].value_counts().head(10))
else:
    print("Received value column not found.")

if received_col and received_col in df.columns:
    sample_value = df[received_col].dropna().unique()
    if 0.001 in sample_value:
        display(df.loc[df[received_col] == 0.001].head())


In [ ]:
delivered_col = column_map.get("delivered")
if delivered_col and delivered_col in df.columns:
    print(f"Analyzing {delivered_col}")
    stats = df[delivered_col].describe()
    display(stats)
    missing_count = df[delivered_col].isna().sum()
    print(f"Missing values: {missing_count} ({missing_count / len(df) * 100:.2f}%)")

    q25 = df[delivered_col].quantile(0.25)
    q75 = df[delivered_col].quantile(0.75)
    iqr = q75 - q25
    lower = q25 - 1.5 * iqr
    upper = q75 + 1.5 * iqr
    outliers = df[(df[delivered_col] < lower) | (df[delivered_col] > upper)]
    print("IQR bounds:", lower, upper)
    print(f"Outliers detected: {len(outliers):,}")
else:
    print("Delivered value column not found.")


In [ ]:
if sns is not None and delivered_col and delivered_col in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    sns.histplot(df[delivered_col].dropna(), bins=50, kde=True, ax=axes[0])
    axes[0].set_title(f"Distribution of {delivered_col}")
    axes[0].set_xlabel(delivered_col)
    sns.boxplot(x=df[delivered_col], ax=axes[1])
    axes[1].set_title(f"Boxplot of {delivered_col}")
    plt.tight_layout()
else:
    print("Install seaborn to render histograms/boxplots.")


In [ ]:
power_zero_col = column_map.get("power_zero")
if power_zero_col and power_zero_col in df.columns:
    print(f"Analyzing {power_zero_col}")
    display(df[power_zero_col].describe())
    print(df[power_zero_col].value_counts(dropna=False))

    if delivered_col and delivered_col in df.columns:
        zero_deliveries = (df[delivered_col] == 0).sum()
        print(f"Zero delivered values: {zero_deliveries:,}")

energy_zero_col = column_map.get("daily_energy_zero")
energy_usage_col = column_map.get("daily_energy")
if energy_zero_col and energy_zero_col in df.columns:
    print(f"Analyzing {energy_zero_col}")
    display(df[energy_zero_col].describe())
    print(df[energy_zero_col].value_counts(dropna=False))

    if energy_usage_col and energy_usage_col in df.columns:
        zero_energy = (df[energy_usage_col] == 0).sum()
        print(f"Zero daily energy usage values: {zero_energy:,}")


In [ ]:
if delivered_col and delivered_col in df.columns:
    corr = df[numeric_columns].select_dtypes(include=[np.number]).corr()
    if sns is not None:
        plt.figure(figsize=(12, 10))
        sns.heatmap(corr, cmap="coolwarm", center=0, annot=False)
        plt.title("Correlation matrix (numeric features)")
        plt.show()
    else:
        print("Install seaborn to render correlation heatmap.")


## 3. Per-Meter Profiling (From `per_meter_eda.py`)


In [ ]:
meter_col = column_map.get("meter")
nmi_col = column_map.get("nmi")

if meter_col and timestamp_col and delivered_col:
    meter_summary = compute_meter_summaries(
        df,
        meter_col=meter_col,
        timestamp_col=timestamp_col,
        delivered_col=delivered_col,
        daily_energy_col=column_map.get("daily_energy"),
        received_col=column_map.get("received"),
        nmi_col=nmi_col,
        power_zero_col=column_map.get("power_zero"),
        daily_energy_zero_col=column_map.get("daily_energy_zero"),
    )
    meter_summary = attach_peer_comparison(
        df,
        meter_summary,
        meter_col=meter_col,
        delivered_col=delivered_col,
        daily_energy_col=column_map.get("daily_energy"),
    )
    print("Per-meter summary (top rows)")
    display(meter_summary.head())
else:
    meter_summary = pd.DataFrame()
    print("Meter, timestamp, or delivered columns are missing; skipping per-meter summary.")


In [ ]:
if not meter_summary.empty:
    print("Summary snapshot")
    display(
        meter_summary[
            [
                col
                for col in [
                    "observation_count",
                    "delivered_mean",
                    "delivered_vs_peers_pct",
                    "power_zero_pct",
                    "coverage_pct",
                ]
                if col in meter_summary.columns
            ]
        ]
        .sort_values(by="delivered_mean", ascending=False)
        .head(10)
    )
else:
    print("Meter summary is empty; nothing to display.")


In [ ]:
if not meter_summary.empty:
    _ = generate_summary_plots(meter_summary, show=True, top_n=10)
else:
    print("Meter summary is empty; skipping plots.")


## 4. Intra-Meter Weekly Diagnostics

This section extends the original scripts by examining how each meter behaves across weeks.


In [ ]:
if meter_col and timestamp_col and value_columns:
    df_weekly = df[[meter_col, timestamp_col] + value_columns].copy()
    df_weekly[timestamp_col] = pd.to_datetime(df_weekly[timestamp_col], errors="coerce")
    df_weekly = df_weekly.dropna(subset=[timestamp_col])
    df_weekly["week_start"] = df_weekly[timestamp_col].dt.to_period("W").apply(lambda p: p.start_time)

    weekly_stats_list = []
    for value_col in value_columns:
        stats = (
            df_weekly.groupby([meter_col, "week_start"], observed=True)[value_col]
            .agg(
                count="count",
                mean="mean",
                median="median",
                std="std",
                min="min",
                q25=lambda x: x.quantile(0.25),
                q75=lambda x: x.quantile(0.75),
                max="max",
            )
            .reset_index()
        )
        stats["value_column"] = value_col
        weekly_stats_list.append(stats)

    weekly_stats = pd.concat(weekly_stats_list, ignore_index=True)
    print("Weekly distribution statistics (sample)")
    display(weekly_stats.head(20))
else:
    weekly_stats = pd.DataFrame()
    print("Cannot compute weekly diagnostics without meter, timestamp, and numeric value columns.")


In [ ]:

if sns is None:
    print("Install seaborn to render weekly diagnostics plots.")
elif weekly_stats.empty or meter_col is None or timestamp_col is None:
    if weekly_stats.empty:
        print("Weekly statistics not computed; skipping plots.")
    else:
        print("Meter or timestamp columns missing; cannot build weekly diagnostics.")
else:
    if not meter_summary.empty and "delivered_sum" in meter_summary.columns:
        top_meters = (
            meter_summary.sort_values("delivered_sum", ascending=False)
            .head(4)
            .index.astype(str)
        )
    elif meter_col in df.columns:
        top_meters = (
            df[meter_col]
            .astype(str)
            .value_counts()
            .head(4)
            .index
        )
    else:
        top_meters = []

    if len(top_meters) == 0:
        print("No meters available to plot.")
    else:
        selected_meters = [str(m) for m in top_meters]

        meter_subset = weekly_stats.copy()
        meter_subset[meter_col] = meter_subset[meter_col].astype(str)
        meter_subset = meter_subset[meter_subset[meter_col].isin(selected_meters)]
        meter_subset["week_start"] = pd.to_datetime(meter_subset["week_start"], errors="coerce")
        meter_subset = meter_subset.dropna(subset=["week_start"])
        meter_subset["week_label"] = meter_subset["week_start"].dt.strftime("%Y-%m-%d")
        if {"mean", "std"}.issubset(meter_subset.columns):
            meter_subset["cv"] = (meter_subset["std"] / meter_subset["mean"]).replace([np.inf, -np.inf], np.nan)
        else:
            meter_subset["cv"] = np.nan

        df_hourly = df[[meter_col, timestamp_col] + value_columns].copy()
        df_hourly = df_hourly[df_hourly[meter_col].astype(str).isin(selected_meters)]
        df_hourly[timestamp_col] = pd.to_datetime(df_hourly[timestamp_col], errors="coerce")
        df_hourly = df_hourly.dropna(subset=[timestamp_col])
        if df_hourly.empty:
            print("No timestamped records available after filtering; skipping hourly diagnostics.")
        else:
            df_hourly[meter_col] = df_hourly[meter_col].astype(str)
            df_hourly["hour"] = df_hourly[timestamp_col].dt.hour
            df_hourly["week_start"] = df_hourly[timestamp_col].dt.to_period("W").apply(lambda p: p.start_time)
            df_hourly["week_label"] = df_hourly["week_start"].dt.strftime("%Y-%m-%d")
            df_hourly["day"] = df_hourly[timestamp_col].dt.date
            df_hourly["day_of_week"] = df_hourly[timestamp_col].dt.day_name()
            hour_order = sorted(df_hourly["hour"].unique())
            week_order = sorted(df_hourly["week_label"].unique())

            daily_hour = (
                df_hourly.groupby([meter_col, "day", "hour"], observed=True)[value_columns]
                .mean()
                .reset_index()
            ) if value_columns else pd.DataFrame()
            if not daily_hour.empty:
                daily_hour[meter_col] = daily_hour[meter_col].astype(str)

            for value_col in value_columns:
                if value_col not in df_hourly.columns:
                    continue

                weekly_data = meter_subset[meter_subset["value_column"] == value_col].copy()
                weekly_data = weekly_data.sort_values([meter_col, "week_start"])

                if not weekly_data.empty:
                    plt.figure(figsize=(10, 4))
                    sns.lineplot(
                        data=weekly_data,
                        x="week_start",
                        y="median",
                        hue=meter_col,
                        marker="o",
                    )
                    plt.title(f"Weekly median {value_col}")
                    plt.xlabel("Week")
                    plt.ylabel(f"Median {value_col}")
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    plt.show()

                    if weekly_data["cv"].notna().any():
                        plt.figure(figsize=(10, 4))
                        sns.lineplot(
                            data=weekly_data.dropna(subset=["cv"]),
                            x="week_start",
                            y="cv",
                            hue=meter_col,
                            marker="o",
                        )
                        plt.title(f"Weekly coefficient of variation for {value_col}")
                        plt.xlabel("Week")
                        plt.ylabel("Coefficient of variation")
                        plt.xticks(rotation=45)
                        plt.tight_layout()
                        plt.show()

                hourly_box = sns.catplot(
                    data=df_hourly[[meter_col, "hour", value_col]].dropna(),
                    x="hour",
                    y=value_col,
                    col=meter_col,
                    kind="box",
                    col_wrap=2,
                    sharey=False,
                    height=4,
                    order=hour_order,
                )
                hourly_box.set_titles("{col_name}")
                hourly_box.set_axis_labels("Hour of day", value_col)
                hourly_box.fig.suptitle(f"Hourly distribution for {value_col}", y=1.03)
                hourly_box.fig.tight_layout()
                plt.show()

                weekly_box = sns.catplot(
                    data=df_hourly[[meter_col, "week_label", value_col]].dropna(),
                    x="week_label",
                    y=value_col,
                    col=meter_col,
                    kind="box",
                    col_wrap=2,
                    sharey=False,
                    height=4,
                    order=week_order,
                )
                weekly_box.set_titles("{col_name}")
                weekly_box.set_axis_labels("Week", value_col)
                for ax in weekly_box.axes.flatten():
                    ax.tick_params(axis="x", rotation=45)
                weekly_box.fig.suptitle(f"Weekly boxplots of hourly {value_col}", y=1.03)
                weekly_box.fig.tight_layout()
                plt.show()

                if not daily_hour.empty and value_col in daily_hour.columns:
                    volatility_box = sns.catplot(
                        data=daily_hour[[meter_col, "hour", value_col]].dropna(),
                        x="hour",
                        y=value_col,
                        col=meter_col,
                        kind="box",
                        col_wrap=2,
                        sharey=False,
                        height=4,
                        order=hour_order,
                    )
                    volatility_box.set_titles("{col_name}")
                    volatility_box.set_axis_labels("Hour of day", f"Daily mean {value_col}")
                    volatility_box.fig.suptitle(f"Hourly volatility (daily means) for {value_col}", y=1.03)
                    volatility_box.fig.tight_layout()
                    plt.show()

                focus_meter = selected_meters[0]
                focus_subset = df_hourly[df_hourly[meter_col] == focus_meter]
                if not focus_subset.empty:
                    heatmap_data = (
                        focus_subset.groupby(["day_of_week", "hour"])[value_col]
                        .std()
                        .unstack(fill_value=np.nan)
                        .reindex(
                            [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ]
                        )
                    )
                    plt.figure(figsize=(12, 4))
                    sns.heatmap(
                        heatmap_data,
                        cmap="mako",
                        linewidths=0.5,
                        linecolor="white",
                        cbar_kws={"label": "Std dev"},
                    )
                    plt.title(f"Day vs hour volatility for {focus_meter} ({value_col})")
                    plt.xlabel("Hour of day")
                    plt.ylabel("Day of week")
                    plt.tight_layout()
                    plt.show()






## Next Steps

- Use the weekly diagnostics above to isolate anomalous weeks per meter.
- Extend the per-meter section with hourly or monthly plots via `hourly_usage_profile` / `monthly_usage_profile` if deeper dives are required.
- Consider exporting `meter_summary` or `weekly_stats` to CSV for further modelling.
